# week 5
## convolutional neural networks CNN
### part 3: visualize the learned filters and feature maps learned by the CNN

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Step 1: Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0 # Normalize pixel values
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Step 2: Define a basic CNN model
def build_cnn():
model = tf.keras.Sequential([
tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=&#39;relu&#39;, input_shape=(28, 28, 1),
name=&#39;conv1&#39;),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation=&#39;relu&#39;, name=&#39;conv2&#39;),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128, activation=&#39;relu&#39;),
tf.keras.layers.Dense(10, activation=&#39;softmax&#39;)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
loss=&#39;categorical_crossentropy&#39;,

metrics=[&#39;accuracy&#39;])
return model

# Step 3: Train and evaluate the CNN
cnn_model = build_cnn()
history = cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 4: Plot training history
plt.plot(history.history[&#39;accuracy&#39;], label=&#39;Train Accuracy&#39;)
plt.plot(history.history[&#39;val_accuracy&#39;], label=&#39;Test Accuracy&#39;)
plt.xlabel(&#39;Epochs&#39;)
plt.ylabel(&#39;Accuracy&#39;)
plt.title(&#39;CNN Training Performance on MNIST&#39;)
plt.legend()
plt.show()

# Step 5: Evaluate the final model
final_accuracy = cnn_model.evaluate(X_test, y_test, verbose=0)[1] * 100
print(f&#39;Final Test Accuracy: {final_accuracy:.2f}%&#39;)

# Step 6: Visualize filters and feature maps
# Ensure the model has been called at least once
_ = cnn_model.predict(np.expand_dims(X_test[0], axis=0))

# Extract outputs of convolutional layers
layer_outputs = [layer.output for layer in cnn_model.layers if &#39;conv&#39; in layer.name]
activation_model = tf.keras.models.Model(inputs=cnn_model.input, outputs=layer_outputs)

# Select a sample image
sample_image = np.expand_dims(X_test[0], axis=0)
feature_maps = activation_model.predict(sample_image)

for layer_name, feature_map in zip([layer.name for layer in cnn_model.layers if &#39;conv&#39; in
layer.name], feature_maps):
plt.figure(figsize=(10, 5))
num_filters = feature_map.shape[-1]
for i in range(min(num_filters, 6)): # Visualizing first 6 feature maps
plt.subplot(1, 6, i + 1)
plt.imshow(feature_map[0, :, :, i], cmap=&#39;viridis&#39;)
plt.axis(&#39;off&#39;)
plt.suptitle(f&#39;Feature Maps - {layer_name}&#39;)
plt.show()

# Step 7: Visualize the filters (kernels) of the first Conv layer
def visualize_filters(layer_name):
layer = cnn_model.get_layer(name=layer_name)
filters, biases = layer.get_weights()

num_filters = filters.shape[-1] # Number of filters
num_channels = filters.shape[-2] # Number of input channels (should be 1 for grayscale)

plt.figure(figsize=(10, 5))
for i in range(min(num_filters, 6)): # Display first 6 filters
plt.subplot(1, 6, i + 1)
plt.imshow(filters[:, :, 0, i], cmap=&#39;gray&#39;)
plt.axis(&#39;off&#39;)
plt.suptitle(f&#39;Filters - {layer_name}&#39;)
plt.show()

visualize_filters(&#39;conv1&#39;) # Visualize the filters of the first Conv layer